In [ ]:
"""
Advanced Tax Assistant System 2025
- Handles Individual and Joint Filings
- Simplified Spouse Handling
"""

from typing import Dict, Literal, Tuple, List
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.FileHandler('tax_system.log'), logging.StreamHandler()]
)

# Type definitions
FilingStatus = Literal['single', 'married', 'joint']
TaxBrackets = Dict[FilingStatus, Tuple[float, ...]]
TaxRates = Tuple[float, ...]

class TaxError(Exception):
    """Base class for tax calculation exceptions"""
    pass

class InvalidIncomeError(TaxError):
    """Raised for invalid income values"""

class TaxConfig:
    """Tax configuration constants for 2025"""
    BRACKETS: TaxBrackets = {
        'single': (0, 11600, 47150, 100525, 191950, 243725),
        'married': (0, 11600, 47150, 100525, 191950, 243725),  # Same as single for MFS
        'joint': (0, 23200, 94300, 201050, 383900, 487450)      # MFJ brackets
    }
    RATES: TaxRates = (0.10, 0.12, 0.22, 0.24, 0.32, 0.35)
    STANDARD_DEDUCTIONS = {
        'single': 14600,
        'married': 14600,  # Married Filing Separately
        'joint': 29200     # Married Filing Jointly
    }
    MEDICAL_FLOOR_PCT = 0.075

class TaxCalculator:
    """Handles progressive tax calculations"""

    def _init_(self, filing_status: FilingStatus):
        self.filing_status = filing_status

    def calculate(self, taxable_income: float) -> float:
        """Calculate tax liability using progressive brackets"""
        if taxable_income < 0:
            raise InvalidIncomeError("Income cannot be negative")

        tax = 0.0
        remaining = taxable_income
        brackets = sorted(TaxConfig.BRACKETS[self.filing_status], reverse=True)

        for bracket, rate in zip(brackets, TaxConfig.RATES):
            if remaining >= bracket:  # Fixed boundary condition
                tax += (remaining - bracket) * rate
                remaining = bracket
        return round(tax, 2)

class DeductionOptimizer:
    """Machine learning model for deduction optimization"""

    def _init_(self):
        self.model = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', DecisionTreeClassifier(
                max_depth=4,
                min_samples_leaf=50,
                class_weight='balanced',
                random_state=42
            ))
        ])
        self._train_model()

    def predict(self, features: np.ndarray) -> bool:
        """Predict optimal deduction strategy"""
        return bool(self.model.predict([features])[0])

    def _train_model(self) -> None:
        """Train model with synthetic data"""
        X = np.array([
            [75_000, 12_000, 2_500, 5_000, 0],    # Single, itemize
            [45_000, 0, 500, 1_000, 0],           # Single, standard
            [150_000, 25_000, 5_000, 15_000, 1],  # Married separate
            [250_000, 50_000, 10_000, 30_000, 2]  # Joint filing
        ])
        y = np.array([1, 0, 1, 1])
        self.model.fit(X, y)

class TaxAssistant:
    """Main tax processing system"""

    def _init_(self):
        self.deduction_optimizer = DeductionOptimizer()

    def process_filings(self, users: List[Dict]) -> List[Dict]:
        """Process multiple tax filings"""
        results = []
        for user in users:
            if user.get('filing_status') == 'joint':
                results.append(self._process_joint_filing(user))
            else:
                results.append(self._process_individual(user))
        return results

    def _process_joint_filing(self, joint_data: Dict) -> Dict:
        """Process joint tax filing"""
        try:
            features = np.array([
                joint_data['combined_income'],
                joint_data['combined_mortgage'],
                joint_data['combined_charity'],
                max(0, joint_data['combined_medical'] -
                    joint_data['combined_income'] * TaxConfig.MEDICAL_FLOOR_PCT),
                2  # Joint filing indicator
            ])

            itemize = self.deduction_optimizer.predict(features)
            deductions = self._calculate_deductions(
                joint_data['combined_income'],
                joint_data['combined_mortgage'],
                joint_data['combined_charity'],
                joint_data['combined_medical'],
                'joint',
                itemize
            )
            taxable_income = max(joint_data['combined_income'] - deductions, 0)

            return {
                'filing_status': 'joint',
                'gross_income': joint_data['combined_income'],
                'deductions': deductions,
                'taxable_income': taxable_income,
                'tax_owed': TaxCalculator('joint').calculate(taxable_income),
                'itemized': itemize
            }
        except KeyError as e:
            logging.error(f"Missing key in joint filing data: {str(e)}")
            raise TaxError("Invalid joint filing data") from e

    def _process_individual(self, user: Dict) -> Dict:
        """Process individual tax filing"""
        self._validate_input(user)

        try:
            features = np.array([
                user['income'],
                user['mortgage_interest'],
                user['charity_donations'],
                max(0, user['medical_expenses'] - user['income'] * TaxConfig.MEDICAL_FLOOR_PCT),
                1 if user['filing_status'] == 'married' else 0
            ])

            itemize = self.deduction_optimizer.predict(features)
            deductions = self._calculate_deductions(
                user['income'],
                user['mortgage_interest'],
                user['charity_donations'],
                user['medical_expenses'],
                user['filing_status'],
                itemize
            )
            taxable_income = max(user['income'] - deductions, 0)

            return {
                'filing_status': user['filing_status'],
                'gross_income': user['income'],
                'deductions': deductions,
                'taxable_income': taxable_income,
                'tax_owed': TaxCalculator(user['filing_status']).calculate(taxable_income),
                'itemized': itemize
            }
        except KeyError as e:
            logging.error(f"Missing key in individual filing: {str(e)}")
            raise TaxError("Invalid individual filing data") from e

    def _calculate_deductions(self, income: float, mortgage: float,
                             charity: float, medical: float,
                             status: str, itemize: bool) -> float:
        """Calculate total deductions"""
        if itemize:
            medical_deductible = max(0, medical - (income * TaxConfig.MEDICAL_FLOOR_PCT))
            itemized_total = mortgage + charity + medical_deductible
            return max(TaxConfig.STANDARD_DEDUCTIONS[status], itemized_total)
        return TaxConfig.STANDARD_DEDUCTIONS[status]

    def _validate_input(self, user: Dict) -> None:
        """Validate user input"""
        required = ['filing_status', 'income', 'mortgage_interest',
                   'charity_donations', 'medical_expenses']
        missing = [field for field in required if field not in user]
        if missing:
            raise ValueError(f"Missing required fields: {missing}")

        if user['filing_status'] not in TaxConfig.BRACKETS:
            raise ValueError(f"Invalid filing status: {user['filing_status']}")

        negative_fields = [field for field in ['income', 'mortgage_interest',
                                              'charity_donations', 'medical_expenses']
                          if user[field] < 0]
        if negative_fields:
            raise ValueError(f"Negative values in fields: {negative_fields}")

def get_validated_input(prompt: str, conv=float, min_val: float = 0) -> float:
    """Get validated numerical input"""
    while True:
        try:
            value = conv(input(prompt))
            if value >= min_val:
                return value
            print(f"Value must be ≥ {min_val}")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

def get_users() -> List[Dict]:
    """Get input for multiple users with joint filing support"""
    filings = []
    num_filings = int(get_validated_input("Number of tax filings: ", conv=int, min_val=1))

    for i in range(num_filings):
        print(f"\n=== Filing {i+1} ===")
        filing_type = input("Filing type (single/married/joint): ").lower().strip()

        if filing_type == 'joint':
            print("\n=== Primary Taxpayer ===")
            primary_income = get_validated_input("Your gross income ($): ")
            primary_mortgage = get_validated_input("Your mortgage interest ($): ")
            primary_charity = get_validated_input("Your charitable donations ($): ")
            primary_medical = get_validated_input("Your medical expenses ($): ")

            print("\n=== Spouse Information ===")
            spouse_income = get_validated_input("Spouse's gross income ($): ")
            spouse_mortgage = get_validated_input("Spouse's mortgage interest ($): ")
            spouse_charity = get_validated_input("Spouse's charitable donations ($): ")
            spouse_medical = get_validated_input("Spouse's medical expenses ($): ")

            filings.append({
                'filing_status': 'joint',
                'combined_income': primary_income + spouse_income,
                'combined_mortgage': primary_mortgage + spouse_mortgage,
                'combined_charity': primary_charity + spouse_charity,
                'combined_medical': primary_medical + spouse_medical
            })
        else:
            income = get_validated_input("Gross income ($): ")
            mortgage = get_validated_input("Mortgage interest ($): ")
            charity = get_validated_input("Charitable donations ($): ")
            medical = get_validated_input("Medical expenses ($): ")

            filings.append({
                'filing_status': filing_type,
                'income': income,
                'mortgage_interest': mortgage,
                'charity_donations': charity,
                'medical_expenses': medical
            })

    return filings

def display_results(results: List[Dict]) -> None:
    """Display tax results with proper formatting"""
    print("\n=== Tax Results ===")
    total_owed = 0.0

    for i, res in enumerate(results, 1):
        print(f"\nFiling {i} ({res['filing_status'].upper()})")
        print(f"Gross Income: ${res['gross_income']:,.2f}")
        print(f"Deductions ({'Itemized' if res['itemized'] else 'Standard'}): ${res['deductions']:,.2f}")
        print(f"Taxable Income: ${res['taxable_income']:,.2f}")
        print(f"Tax Owed: ${res['tax_owed']:,.2f}")
        total_owed += res['tax_owed']

    print("\n=== Summary ===")
    print(f"Total Tax Liability: ${total_owed:,.2f}")

if __name__ == "__main__": # Changed _name_ to __name__
    try:
        users = get_users()
        results = TaxAssistant().process_filings(users)
        display_results(results)
    except (TaxError, ValueError) as e:
        print(f"\nProcessing Error: {str(e)}")
    except Exception as e:
        print(f"\nUnexpected Error: {str(e)}")

Number of tax filings: 10

=== Filing 1 ===
